In [1]:
# Create virtual enviroment to install sklearn development version 0.21.2 as there are memory 
# issues with sklearn current version 0.21.1
conda activate myenvsl


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import check_classification_targets
from joblib import Parallel, delayed

class HQC(BaseEstimator, ClassifierMixin):
    """The Helstrom Quantum Centroid (HQC) classifier is a quantum-inspired supervised 
    classification approach for data with binary classes (ie. data with 2 classes only).
                         
    Parameters
    ----------
    rescale : int or float, default = 1
        The dataset rescaling factor. A parameter used for rescaling the dataset. 
    n_copies : int, default = 1
        The number of copies to take for each quantum density. This is equivalent to taking 
        the n-fold Kronecker tensor product for each quantum density.  
    n_jobs : int, default = None
        The number of CPU cores used when parallelizing. If -1 all CPUs are used. If 1 is given, 
        no parallel computing code is used at all. For n_jobs below -1, (n_cpus + 1 + n_jobs) 
        are used. Thus for n_jobs = -2, all CPUs but one are used. None is a marker for ‘unset’ 
        that will be interpreted as n_jobs = 1.
    n_splits : int, default = 1
        The number of subset splits performed on the input dataset row-wise and on the number 
        of eigenvalues/eigenvectors of the Quantum Helstrom observable for optimal speed 
        performance. For optimal speed, recommend using n_splits = int(numpy.ceil(Number of 
        CPU cores used/2)). If memory blow-out occurs, reduce n_splits.
    
    Attributes
    ----------
    classes_ : ndarray, shape (2,)
        Sorted binary classes.
    centroid_ : ndarray, shape (2, n_features + 1, n_features + 1)
        Quantum Centroids for class with index 0 and 1 respectively.
    q_Hels_obs_ : ndarray, shape (n_features + 1, n_features + 1)
        Quantum Helstrom observable.
    proj_sum_ : tuple, shape (2, n_features + 1, n_features + 1)
        Sum of the projectors of the Quantum Helstrom observable's eigenvectors, which has
        corresponding positive and negative eigenvalues respectively.
    Hels_bound_ : float
        Helstrom bound is the upper bound of the probability that one can correctly 
        discriminate whether a quantum density is of which of the two binary quantum density 
        pattern.          
    """
    # Added binary_only tag as required by sklearn check_estimator
    def _more_tags(self):
        return {'binary_only': True}
    
    
    # Initialize model hyperparameters
    def __init__(self, rescale=1, n_copies=1, n_jobs=None, n_splits=1):
        self.rescale = rescale
        self.n_copies = n_copies
        self.n_jobs = n_jobs
        self.n_splits = n_splits
        
    
    # Function for fit
    def fit(self, X, y):
        """Perform HQC classification with the inverse of the standard stereographic 
        projection encoding, with the option to rescale the dataset prior to encoding.
                
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples. An array of int or float.
        y : array-like, shape (n_samples,)
            The training input binary target values. An array of str, int or float.
            
        Returns
        -------
        self : object
            Returns self.
        """
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        
        # Ensure target y is of non-regression type  
        # Added as required by sklearn check_estimator
        check_classification_targets(y)
    
        # Store binary classes and encode y into binary class indexes 0 and 1
        self.classes_, y_class_index = np.unique(y, return_inverse = True)
        
        # Cast X to float to ensure all following calculations below are done in float  
        # rather than integer
        X = X.astype(float)
        
        # Rescale X
        X = self.rescale*X
        
        # Calculate sum of squares of each row (sample) in X
        X_sq_sum = (X**2).sum(axis = 1)
        
        # Number of rows in X
        m = X.shape[0]
        
        # Number of columns in X
        n = X.shape[1]
        
        # Function to calculate X'    
        def X_prime_func(i):
            return (1 / (X_sq_sum[i] + 1)) \
                   *(np.concatenate((2*X, (X_sq_sum - 1).reshape(-1, 1)), axis = 1)[i, :])
        
        # Calculate X'
        X_prime = np.array(Parallel(n_jobs = self.n_jobs) \
                          (delayed(X_prime_func)(i) for i in range(m)))
        
        # Function to calculate terms in the Quantum Centroids and quantum Helstrom 
        # observable for each class
        def centroid_terms_func(i):
            # Determine rows (samples) in X' belonging to either class
            X_prime_class = X_prime[y_class_index == i]
            
            # Number of rows (samples) in X' belonging to either class
            M_class = X_prime_class.shape[0]
            
            # Split X' belonging to either class into n_splits subsets, row-wise
            X_prime_class_split = np.array_split(X_prime_class, 
                                                 indices_or_sections = self.n_splits, 
                                                 axis = 0)
            
            # Function to calculate terms in the Quantum Centroids and quantum Helstrom
            # observable for each class, per subset split
            def X_prime_class_split_func(j):
                # Counter for j-th split X' belonging to either class
                X_prime_class_split_jth = X_prime_class_split[j]
                
                # Number of rows (samples) in j-th split X' belonging to either class
                M_class_split = X_prime_class_split_jth.shape[0]
            
                # Number of rows/columns in density matrix
                density_nrow_ncol = (n + 1)**self.n_copies
            
                # Initialize arrays density_sum and centroid
                density_sum = np.zeros((density_nrow_ncol, density_nrow_ncol))
                centroid = density_sum
                for k in range(M_class_split):
                    # Encode into quantum densities using the inverse of the standard 
                    # stereographic projection encoding method
                    X_prime_class_split_each_row = X_prime_class_split_jth[k, :]
                    density_each_row = np.dot(X_prime_class_split_each_row.reshape(-1, 1),
                                              X_prime_class_split_each_row.reshape(1, -1))
                
                    # Calculate n-fold Kronecker tensor product
                    if self.n_copies == 1:
                        density_each_row = density_each_row
                    else:
                        density_each_row_copy = density_each_row
                        for u in range(self.n_copies - 1):
                            density_each_row = np.kron(density_each_row, density_each_row_copy)
                
                    # Calculate sum of quantum densities belonging to either class
                    density_sum = density_sum + density_each_row
                
                    # Calculate Quantum Centroid
                    # Added ZeroDivisionError as required by sklearn check_estimator
                    try:
                        centroid = (1 / M_class)*density_sum
                    except ZeroDivisionError:
                        centroid = 0
                return M_class_split, centroid, (1 / m)*density_sum        
            return np.sum(Parallel(n_jobs = self.n_jobs) \
                         (delayed(X_prime_class_split_func)(j) for j in range(self.n_splits)), axis = 0)
            
        # Calculate Quantum Centroids and terms in the quantum Helstrom observable
        centroid_terms = np.array(Parallel(n_jobs = self.n_jobs) \
                                          (delayed(centroid_terms_func)(i) for i in range(2)))
        
        # Determine Quantum Centroids
        self.centroid_ = centroid_terms[:, 1]
           
        # Calculate quantum Helstrom observable
        self.q_Hels_obs_ = centroid_terms[0, 2] - centroid_terms[1, 2]     
        
        # Calculate eigenvalues w and eigenvectors v of the quantum Helstrom observable
        w, v = np.linalg.eigh(self.q_Hels_obs_)
        
        # Length of w
        len_w = len(w)
        
        # Initialize array eigval_class
        eigval_class = np.empty_like(w)
        for i in range(len_w):
            # Create an array of 0s and 1s to indicate positive and negative eigenvalues
            # respectively
            if w[i] > 0:
                eigval_class[i] = 0
            else:
                eigval_class[i] = 1
        
        # Transpose matrix v containing eigenvectors to row-wise
        eigvec = v.T
        
        # Function to calculate sum of the projectors corresponding to positive and negative
        # eigenvalues respectively
        def sum_proj_func(i):
            # Determine eigenvectors belonging to positive and negative eigenvalues respectively
            eigvec_class = eigvec[eigval_class == i]
            
            # Split eigenvectors into n_splits subsets
            eigvec_class_split = np.array_split(eigvec_class, 
                                                indices_or_sections = self.n_splits, 
                                                axis = 0)
            
            # Function to calculate sum of the projectors corresponding to positive and negative
            # eigenvalues respectively, per subset split
            def eigvec_class_split_func(j):
                # Initialize array proj_sum_split
                proj_sum_split = np.zeros_like(self.q_Hels_obs_)
                for k in eigvec_class_split[j]:
                    # Calculate sum of the projectors corresponding to positive and negative
                    # eigenvalues respectively, per subset split
                    proj_sum_split = proj_sum_split + np.dot(k.reshape(-1, 1), k.reshape(1, -1))
                return proj_sum_split        
            return np.sum(Parallel(n_jobs = self.n_jobs) \
                         (delayed(eigvec_class_split_func)(j) for j in range(self.n_splits)), axis = 0)
        
        # Calculate sum of the projectors corresponding to positive and negative eigenvalues
        # respectively
        self.proj_sum_ = Parallel(n_jobs = self.n_jobs) \
                         (delayed(sum_proj_func)(i) for i in range(2))    
                       
        # Calculate Helstrom bound
        self.Hels_bound_ = (centroid_terms[0, 0] / m)*np.einsum('ij,ji->', self.centroid_[0], 
                                                                self.proj_sum_[0]) \
                           + (centroid_terms[1, 0] / m)*np.einsum('ij,ji->', self.centroid_[1], 
                                                                  self.proj_sum_[1])
        return self
        
    
    # Function for predict_proba
    def predict_proba(self, X):
        """Performs HQC classification on X and returns the trace of the dot product of the densities 
        and the sum of the projectors with corresponding positive and negative eigenvalues respectively.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.       
            
        Returns
        -------
        trace_matrix : array-like, shape (n_samples, 2)
            Column index 0 corresponds to the trace of the dot product of the densities and the sum  
            of projectors with positive eigenvalues. Column index 1 corresponds to the trace of the  
            dot product of the densities and the sum of projectors with negative eigenvalues. An array 
            of float.
        """
        # Check if fit had been called
        check_is_fitted(self, ['proj_sum_'])

        # Input validation
        X = check_array(X)
        
        # Cast X to float to ensure all following calculations below are done in float 
        # rather than integer
        X = X.astype(float)        
        
        # Rescale X
        X = self.rescale*X        
        
        # Calculate sum of squares of each row (sample) in X
        X_sq_sum = (X**2).sum(axis = 1)
        
        # Number of rows in X
        m = X.shape[0]
        
        # Number of columns in X
        n = X.shape[1]

        # Function to calculate X'    
        def X_prime_func(i):
            return (1 / (X_sq_sum[i] + 1)) \
                   *(np.concatenate((2*X, (X_sq_sum - 1).reshape(-1, 1)), axis=1)[i, :])
        
        # Calculate X'
        X_prime = np.array(Parallel(n_jobs = self.n_jobs) \
                          (delayed(X_prime_func)(i) for i in range(m)))        
        
        # Function to calculate trace values for each class
        def trace_func(i):
            # Split X' into n_splits subsets, row-wise
            X_prime_split = np.array_split(X_prime, 
                                           indices_or_sections = self.n_splits, 
                                           axis = 0)
            
            # Function to calculate trace values for each class, per subset split
            def trace_split_func(j):
                # Counter for j-th split X'
                X_prime_split_jth = X_prime_split[j]
                
                # Number of rows (samples) in j-th split X'
                X_prime_split_m = X_prime_split_jth.shape[0]
                
                # Initialize array trace_class_split
                trace_class_split = np.empty(X_prime_split_m)
                for k in range(X_prime_split_m):
                    # Encode into quantum densities using the inverse of the standard stereographic
                    # projection encoding method
                    X_prime_split_each_row = X_prime_split_jth[k, :]
                    density_each_row = np.dot(X_prime_split_each_row.reshape(-1, 1), 
                                              X_prime_split_each_row.reshape(1, -1))
                
                    # Calculate n-fold Kronecker tensor product
                    if self.n_copies == 1:     
                        density_each_row = density_each_row
                    else:
                        density_each_row_copy = density_each_row
                        for u in range(self.n_copies - 1):
                            density_each_row = np.kron(density_each_row, density_each_row_copy)
                        
                    # Calculate trace of the dot product of density of each row and sum of projectors 
                    # with corresponding positive and negative eigenvalues respectively    
                    trace_class_split[k] = np.einsum('ij,ji->', density_each_row, self.proj_sum_[i])
                return trace_class_split
            
            # Calculate trace values for each class, per subset split
            trace_class = Parallel(n_jobs = self.n_jobs) \
                          (delayed(trace_split_func)(j) for j in range(self.n_splits))
            return np.concatenate(trace_class, axis = 0)
            
        # Calculate trace values for each class
        trace_matrix = np.transpose(Parallel(n_jobs = self.n_jobs) \
                                   (delayed(trace_func)(i) for i in range(2)))
        return trace_matrix
        
    
    # Function for predict
    def predict(self, X):
        """Performs HQC classification on X and returns the binary classes.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.
            
        Returns
        -------
        self.classes_[predict_trace_index] : array-like, shape (n_samples,)
            The predicted binary classes. An array of str, int or float.
        """
        # Determine column index with the higher trace value in trace_matrix
        # If both columns have the same trace value, returns column index 0
        predict_trace_index = np.argmax(self.predict_proba(X), axis = 1)
        # Returns the predicted binary classes
        return self.classes_[predict_trace_index]

In [2]:
# appendicitis dataset (7 features, 106 rows)
import pandas as pd

df = pd.read_csv('appendicitis.tsv',delimiter='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

In [3]:
# Check F1 score and Helstrom bound values for various rescale and n_copies values
model = HQC(rescale=0.5, n_copies=3, n_jobs=-1, n_splits=2).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted'), model.Hels_bound_

(0.7520661157024794, 0.8772542482734037)

In [8]:
# Time required for n_copies=1
%timeit HQC(rescale=0.5, n_copies=1, n_jobs=-1, n_splits=2).fit(X_train, y_train)

490 ms ± 77.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# Time required for n_copies=2
%timeit HQC(rescale=0.5, n_copies=2, n_jobs=-1, n_splits=2).fit(X_train, y_train)

499 ms ± 52.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# Time required for n_copies=3
%timeit HQC(rescale=0.5, n_copies=3, n_jobs=-1, n_splits=2).fit(X_train, y_train)

2.14 s ± 84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# Time required for n_copies=4
%timeit HQC(rescale=0.5, n_copies=4, n_jobs=-1, n_splits=2).fit(X_train, y_train)

10min 50s ± 32.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# Time required for n_copies=5. Memory blow-out
%timeit HQC(rescale=0.5, n_copies=5, n_jobs=-1, n_splits=2).fit(X_train, y_train)

MemoryError: Unable to allocate 8.00 GiB for an array with shape (32768, 32768) and data type float64

In [6]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [7]:
# Memory required for n_copies=1
%memit HQC(rescale=0.5, n_copies=1, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 119.21 MiB, increment: 0.08 MiB


In [9]:
# Memory required for n_copies=2
%memit HQC(rescale=0.5, n_copies=2, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 120.38 MiB, increment: 0.03 MiB


In [10]:
# Memory required for n_copies=3
%memit HQC(rescale=0.5, n_copies=3, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 136.98 MiB, increment: 16.61 MiB


In [23]:
# banana dataset (2 features, 5300 rows)
import pandas as pd

df = pd.read_csv('banana.tsv', sep='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

In [24]:
# Check F1 score and Helstrom bound values for various rescale and n_copies values
model = HQC(rescale=0.5, n_copies=4, n_jobs=-1, n_splits=2).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted'), model.Hels_bound_

(0.858978398722441, 0.7732939055876817)

In [14]:
# Time required for n_copies=1
%timeit HQC(rescale=0.5, n_copies=1, n_jobs=-1, n_splits=2).fit(X_train, y_train)

1.21 s ± 91.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
# Time required for n_copies=2
%timeit HQC(rescale=0.5, n_copies=2, n_jobs=-1, n_splits=2).fit(X_train, y_train)

1.5 s ± 45.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
# Time required for n_copies=3
%timeit HQC(rescale=0.5, n_copies=3, n_jobs=-1, n_splits=2).fit(X_train, y_train)

1.86 s ± 83.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
# Time required for n_copies=4
%timeit HQC(rescale=0.5, n_copies=4, n_jobs=-1, n_splits=2).fit(X_train, y_train)

2.77 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
# Time required for n_copies=5
%timeit HQC(rescale=0.5, n_copies=5, n_jobs=-1, n_splits=2).fit(X_train, y_train)

6.18 s ± 131 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
# Time required for n_copies=6
%timeit HQC(rescale=0.5, n_copies=6, n_jobs=-1, n_splits=2).fit(X_train, y_train)

57.1 s ± 944 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [125]:
# Time required for n_copies=7
%timeit HQC(rescale=0.5, n_copies=7, n_jobs=-1, n_splits=2).fit(X_train, y_train)

9min 23s ± 12.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [16]:
# Memory required for n_copies=1
%memit HQC(rescale=0.5, n_copies=1, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 109.55 MiB, increment: 0.00 MiB


In [17]:
# Memory required for n_copies=2
%memit HQC(rescale=0.5, n_copies=2, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 109.30 MiB, increment: 0.00 MiB


In [18]:
# Memory required for n_copies=3
%memit HQC(rescale=0.5, n_copies=3, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 109.30 MiB, increment: 0.00 MiB


In [19]:
# Memory required for n_copies=4
%memit HQC(rescale=0.5, n_copies=4, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 110.30 MiB, increment: 1.00 MiB


In [20]:
# Memory required for n_copies=5
%memit HQC(rescale=0.5, n_copies=5, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 114.02 MiB, increment: 3.71 MiB


In [25]:
# Memory required for n_copies=6
%memit HQC(rescale=0.5, n_copies=6, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 127.69 MiB, increment: 42.93 MiB


In [22]:
# Memory required for n_copies=7
%memit HQC(rescale=0.5, n_copies=7, n_jobs=-1, n_splits=2).fit(X_train, y_train)

peak memory: 493.77 MiB, increment: 382.32 MiB
